In [197]:

import pandas as pd
import pandas.testing as pd_testing
import pyodbc
import pycountry
import unittest


PLblOutPath = 'C:\\Users\\belle\\PycharmProjects\\ContinentalProject\\Data\Papaya\\papaya_output_labels.csv'
PImgOutPath = 'C:\\Users\\belle\\PycharmProjects\\ContinentalProject\\Data\Papaya\\papaya_output_images.csv'
KuOutPath = 'C:\\Users\\belle\\PycharmProjects\\ContinentalProject\\Data\Kumquat\\kumquat_output.csv'


# Connection string to MSSQL db using Azure Data Studio
conn = pyodbc.connect('Driver={SQL Server};'
     'Server=.;'
     'Database=Continental;'
     'Trusted_Connection=yes;')

# Create {country : alpha3} dict
countryMap = {}

for country in pycountry.countries:
    countryMap[country.name] = country.alpha_3


def lbl_sizeToNumeric(df):
     df[['label_size']] = df[['label_size']].replace(',', '.', regex=True)


def labelDeltaLoaderUpdater(df, cursor):
    for index, row in df.iterrows():
     try:
        # Check if "record_file_name" or "timestamp" is empty
        if not pd.isna(row.record_file_name) and not pd.isna(row.timestamp) and not pd.isna(row.label_coordinates):
          # Check if entry already exists in db
          cursor.execute("SELECT COUNT(*) FROM dbo.labels WHERE record_file_name=? AND timestamp=?",
                       (row.record_file_name, row.timestamp))
          count = cursor.fetchone()[0]

          if count == 0:
            # Entry doesn't exist, insert the entry into the db
            cursor.execute("INSERT INTO dbo.labels (record_file_name, timestamp, label_type, label_size, label_coordinates, version) VALUES (?, ?, ?, ?, ?, ?)",
                           (row.record_file_name, row.timestamp, row.label_type, row.label_size, row.label_coordinates, row.version))
          else:
            # Entry exists, update the entry in the db
            cursor.execute("UPDATE dbo.labels SET label_type=?, label_size=?, label_coordinates=?, version=? WHERE record_file_name=? AND timestamp=?",
                           (row.label_type, row.label_size, row.label_coordinates, row.version, row.record_file_name, row.timestamp))
     except Exception as e:
          print(f"An error occurred while processing row {index}: {str(e)}")


def imageDeltaLoader(df, cursor):
    # Create a Series with country names mapped to alpha3 codes
    countrySeries = df['country'].map(country_map)

    for index, row in df.iterrows():
     try:
        # Check if "record_file_name" or "timestamp" is empty
        if not pd.isna(row.record_file_name) and not pd.isna(row.timestamp):
          # Check if entry already exists in db
          cursor.execute("SELECT COUNT(*) FROM dbo.images WHERE record_file_name=? AND timestamp=?", (row.record_file_name, row.timestamp))
          if cursor.fetchone()[0] == 0:  # Entry doesn't exist
            # Insert the entry into the db
            cursor.execute("INSERT INTO dbo.images (record_file_name, timestamp, image_format, image_path, country) VALUES (?, ?, ?, ?, ?)",
                           (row.record_file_name, row.timestamp, row.image_format, row.image_path, countrySeries[index]))
     except Exception as e:
          print(f"An error occurred while processing row {index}: {str(e)}")




class PapayaLoader:
     def __init__(self, imgPath, lblPath):
          self.imgPath = imgPath
          self.lblPath = lblPath
     def load(self):
          self.PImgOut = pd.read_csv(self.imgPath, delimiter = ';')
          self.PLblOut = pd.read_csv(self.lblPath, delimiter = ';')
          lbl_sizeToNumeric(self.PLblOut)
          cursor = conn.cursor()
          labelDeltaLoaderUpdater(self.PLblOut, cursor)
          imageDeltaLoader(self.PImgOut, cursor)
          conn.commit()
          cursor.close()


class KumquatLoader:
     def __init__(self, path):
          self.path = path
     def load(self):
          self.KuOut = pd.read_csv(self.path, delimiter = ';')
          lbl_sizeToNumeric(self.KuOut)
          self.KuImgOut = self.KuOut[['record_file_name', 'timestamp', 'image_format', 'image_path', 'country']]
          self.KuLblOut = self.KuOut[['record_file_name', 'timestamp', 'label_type', 'label_size', 'label_coordinates', 'version']]
          cursor = conn.cursor()
          labelDeltaLoaderUpdater(self.KuLblOut, cursor)
          imageDeltaLoader(self.KuImgOut, cursor)
          conn.commit()
          cursor.close()

In [196]:
# Testing
# I would've also liked to test if I've inserted the correct number of rows into the db by using a mock df which has invalid records but I'll need to sit down and learn how to implement a mock cursor for that.
class PapayaLoaderTestCase(unittest.TestCase):
    def setUp(self):
        self.imgPath = 'C:\\Users\\belle\\PycharmProjects\\ContinentalProject\\Data\\Papaya\\papaya_output_images.csv'
        self.lblPath = 'C:\\Users\\belle\\PycharmProjects\\ContinentalProject\\Data\\Papaya\\papaya_output_labels.csv'

    def test_loadFromCsv(self):
        papayaLoader = PapayaLoader(self.imgPath, self.lblPath)
        papayaLoader.load()

        self.assertIsInstance(papayaLoader.PImgOut, pd.DataFrame)
        self.assertIsInstance(papayaLoader.PLblOut, pd.DataFrame)

    def test_lbl_sizeToNumeric(self):
        # Test lbl_sizeToNumeric
        df = pd.DataFrame({'label_size': ['1', '3,4', '5,6']})
        # Expected DataFrame after applying lbl_sizeToNumeric
        expected_df = pd.DataFrame({'label_size': ['1', '3.4', '5.6']})
        lbl_sizeToNumeric(df)
        # Assert that the resulting DataFrame matches the expected DataFrame
        pd_testing.assert_frame_equal(df, expected_df)

    

        
  


class KumquatLoaderTestCase(unittest.TestCase):
    def setUp(self):
        self.path = 'C:\\Users\\belle\\PycharmProjects\\ContinentalProject\\Data\\Kumquat\\kumquat_output.csv'

    def test_loadFromCsv(self):
        kumquatLoader = KumquatLoader(self.path)
        kumquatLoader.load()

        self.assertIsInstance(kumquatLoader.KuOut, pd.DataFrame)
        self.assertIsInstance(kumquatLoader.KuImgOut, pd.DataFrame)
        self.assertIsInstance(kumquatLoader.KuLblOut, pd.DataFrame)

    def test_lbl_sizeToNumeric(self):
        df = pd.DataFrame({'label_size': ['1', '3,4', '5,6']})
        # Expected DataFrame after applying lbl_sizeToNumeric
        expected_df = pd.DataFrame({'label_size': ['1', '3.4', '5.6']})
        lbl_sizeToNumeric(df)
        # Assert that the resulting DataFrame matches the expected DataFrame
        pd_testing.assert_frame_equal(df, expected_df)




# Run tests
unittest.main(argv=[''], verbosity=2, exit=False)

test_lbl_sizeToNumeric (__main__.KumquatLoaderTestCase) ... 

ok
test_loadFromCsv (__main__.KumquatLoaderTestCase) ... 

ok
test_lbl_sizeToNumeric (__main__.PapayaLoaderTestCase) ... 

ok
test_loadFromCsv (__main__.PapayaLoaderTestCase) ... 

ok

----------------------------------------------------------------------
Ran 4 tests in 0.118s

OK


In [198]:
# Load
LoadPapaya = PapayaLoader(PImgOutPath, PLblOutPath)
LoadPapaya.load()

LoadKumquat = KumquatLoader(KuOutPath)
LoadKumquat.load()